
## Initial testing on a naive bayes classifier yielded about 56% accuracy. 
 we used a reduced set to train the model  due to time constraints. 
 
 further explorations will attempt to improve on this baseline  

In [1]:
import joblib
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import logging
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

##get the data into pandas for analysis\"\"\"\n",
df = pd.read_csv('rspct.tsv', sep='\t')



reduced = df.sample(frac=.3)


In [2]:
reduced = pd.read_csv('reduced30k.tsv','\t')

reduced.to_csv('reduced30k.tsv', sep='\t')

In [3]:
labels = reduced['subreddit'].drop_duplicates()


In [4]:
reduced.shape

(303900, 5)

classes have been balanced! time to clean some of the punctation, and pseudo html from the text


In [5]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /home/dliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
df = reduced

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    return text


df['selftext'] = df['selftext'].apply(clean_text)
df['title'] = df['title'].apply(clean_text)


FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [7]:
!pip install bs4


In [ ]:
df.to_csv('cleaned_text.csv', sep='\t\')

In [ ]:
cln_red = df.sample(frac=.3, random_state=42)

In [ ]:
cln_red.shape

In [ ]:
cln_red.to_csv('cleaned_reduced.tsv', sep='\t')

In [ ]:
%cd /home/dliu/lambda/build/notebooks



In [ ]:
df = pd.read_csv('cleaned_reduced.tsv', sep='\t')

In [ ]:
cln_red = df.sample(frac=.5, random_state=42)

At this point we can consider if we want to prepend the title to the post, or handel it as a separate feature. i think for simplicity, the first time, we will just use the post only then compare to results with title + selftext 

In [ ]:
from sklearn.model_selection import train_test_split
X = cln_red.selftext
y = cln_red.subreddit
X_train,  X_test, y_train,y_test = train_test_split(X,y,test_size=.2,random_state=42)

In [ ]:
# setup a naive bayesian classifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# setup pipelinen and fit
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

%%time

In [ ]:
my_labels = cln_red['subreddit'].drop_duplicates()

In [ ]:

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
nb_class_rep = classification_report(y_test, y_pred,target_names=my_labels)

In [ ]:
print(nb_class_rep)

## lets save this model as a fallback incase we have problems


In [ ]:
import pickle
with open('nb_selftext.pkl', 'wb') as f:
    pickle.dump(nb, f, pickle.HIGHEST_PROTOCOL)

We will use the following to load this model elsewhere:

    with open('nb_self', 'rb') as handle:
        model = pickle.load(handle)
    
    model
